# Imports

In [53]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import timeit


# Reproducibilidad
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

from Ising_Functions import *

In [54]:
def compara_estados(red1, red2, metodo=None):
    """
    Función para comparar dos estados de la red o dos redes.
    Toma ambas redes.
    Devuelve la suma de las diferencias dividida por el tamaño de la red
    """
    resultado = red1 - red2
    return np.mean(resultado/2)

# Paths

In [55]:
test_mode = True

if test_mode:
    print("Test mode activado")
    test_dir = Path("test")
    test_dir.mkdir(exist_ok=True)
    figures_dir = Path("test") / "figures"
    figures_dir.mkdir(exist_ok=True)
    data_dir = Path("test") / "data"
    data_dir.mkdir(exist_ok=True)
    gifs_dir = Path("test") / "gifs"
    gifs_dir.mkdir(exist_ok=True)

else:
    # Paths para guardar imágenes y resultados
    figures_dir = Path("figures")
    figures_dir.mkdir(exist_ok=True)  # Crear directorio si no existe

    data_dir = Path("data")
    data_dir.mkdir(exist_ok=True)  # Crear directorio si no existe

    gifs_dir = Path("gifs")
    gifs_dir.mkdir(exist_ok=True)  # Crear directorio si no existe

Test mode activado


# Control Variables

In [56]:
# Configuración de la simulación
N = 64                # Tamaño de la red
T = 0.75              # Temperatura (en unidades arbitrarias)
pasos = 100           # Número de pasos Monte Carlo
num_imagenes = 5      # Número de imágenes a guardar para el paper



if test_mode:
    N = 64
    T = 1
    pasos = 150
    num_imagenes = 5

# Variables de control comunes
generar_gif = True
intervalo_gif = 1
calcular_magnetizacion = True
intervalo_magnetizacion = 1
verbose = False

# Opción de modelo: "clasico", "modificado" o "ambos"
modelo = "ambos"  # Cambia este valor segun la simulación deseada

# Simulations

## Update boundaries each pMC

### Modif

In [57]:
if modelo == "modificado" or modelo == "ambos":
    if verbose:
        print(f"Simulando Modelo de Ising modificado (con interacciones de next-neighbours) para T = {T}, N = {N}...")
    frames_mod, mags_mod, pasos_MC_frames_mod, pasos_MC_mags_mod = ising_simulacion_modificado(
        N, T, pasos=pasos,
        generar_gif=generar_gif, intervalo_gif=intervalo_gif,
        calcular_magnetizacion=calcular_magnetizacion, intervalo_magnetizacion=intervalo_magnetizacion
    )

    if generar_gif:
        gif_name_mod = f"ising_mod_T{T}_N{N}.gif"
        if verbose:
            print(f"Guardando animación del modelo modificado como {gif_name_mod}...")
        crear_gif(frames_mod, nombre=gif_name_mod, dir_path=gifs_dir)

    if calcular_magnetizacion:
        mag_filepath_mod = data_dir / f"magnetizacion_mod_T{T}_N{N}.txt"
        if verbose:
            print(f"Guardando magnetización promedio (modelo modificado) en {mag_filepath_mod}...")
        np.savetxt(mag_filepath_mod, mags_mod)
        if verbose:
            print("Magnetización (modificado) guardada correctamente.")

        curva_magnetizacion_mod = f"magnetizacion_curva_mod_T{T}_N{N}.png"
        graficar_magnetizacion(mags_mod, pasos_MC_mags_mod, T, N, nombre=curva_magnetizacion_mod, dir_path=figures_dir)
        if verbose:
            print(f"Curva de magnetización (modificado) guardada como {curva_magnetizacion_mod}.")

    if verbose:
        print("Guardando imágenes equiespaciadas (modelo modificado) para el paper...")
    guardar_imagenes(frames_mod, num_imagenes=num_imagenes, nombre_base=f"ising_mod_T{T}_N{N}", dir_path=figures_dir, verbose=verbose)

### Classic

In [58]:
if modelo == "clasico" or modelo == "ambos":
    if verbose:
        print(f"\nSimulando Modelo de Ising clásico para T = {T}, N = {N}...")
    frames_cls, mags_cls, pasos_MC_frames_cls, pasos_MC_mags_cls = ising_simulacion_clasico(
        N, T, pasos=pasos,
        generar_gif=generar_gif, intervalo_gif=intervalo_gif,
        calcular_magnetizacion=calcular_magnetizacion, intervalo_magnetizacion=intervalo_magnetizacion
    )

    if generar_gif:
        gif_name_cls = f"ising_cls_T{T}_N{N}.gif"
        if verbose:
            print(f"Guardando animación del modelo clásico como {gif_name_cls}...")
        crear_gif(frames_cls, nombre=gif_name_cls, dir_path=gifs_dir)

    if calcular_magnetizacion:
        mag_filepath_cls = data_dir / f"magnetizacion_cls_T{T}_N{N}.txt"
        if verbose:
            print(f"Guardando magnetización promedio (modelo clásico) en {mag_filepath_cls}...")
        np.savetxt(mag_filepath_cls, mags_cls)
        if verbose:
            print("Magnetización (clásico) guardada correctamente.")

        curva_magnetizacion_cls = f"magnetizacion_curva_cls_T{T}_N{N}.png"
        graficar_magnetizacion(mags_cls, pasos_MC_mags_cls, T, N, nombre=curva_magnetizacion_cls, dir_path=figures_dir)
        if verbose:
            print(f"Curva de magnetización (clásico) guardada como {curva_magnetizacion_cls}.")

    if verbose:
        print("Guardando imágenes equiespaciadas (modelo clásico) para el paper...")
    guardar_imagenes(frames_cls, num_imagenes=num_imagenes, nombre_base=f"ising_cls_T{T}_N{N}", dir_path=figures_dir, verbose=verbose)
if verbose:
    print("\nSimulación completada. Resultados guardados correctamente.")

In [59]:
compara_estados(frames_mod[0], frames_cls[0])

0.00146484375

In [70]:
def crear_matriz_clasico_toroidal(N, values=[1, -1], padding_width=1, mode='wrap', seed=42):
    """
    Crea una red de NxN spins aleatorios (valores +1 o -1) y añade bordes periódicos toroidales.
    
    Parámetros:
    - N (int): Tamaño de la red (NxN).
    - padding_width (int): Ancho del borde añadido (por defecto 1).
    - mode (str): Modo de relleno para los bordes (por defecto 'wrap' para bordes periódicos).

    Devuelve:
    - numpy.ndarray: Red con bordes periódicos añadidos.
    """

    rng_generator = np.random.default_rng(seed=seed)
    
    red = rng_generator.choice(values, size=(N, N))
    web = np.pad(red, pad_width=padding_width, mode=mode)

    return web